## Setup notebook

In [1]:
!pip install fasttext

    100% |████████████████████████████████| 81kB 4.3MB/s ta 0:00:011
  Running setup.py bdist_wheel for fasttext ... done
  Stored in directory: /home/dsaez/.cache/pip/wheels/73/8e/5d/ecb50b90adaab5868ae1d8df180f31e55e85c2f055aaf2fb35
Successfully built fasttext


In [2]:
!git clone https://github.com/Babylonpartners/fastText_multilingual.git

Cloning into 'fastText_multilingual'...
remote: Counting objects: 226, done.
remote: Total 226 (delta 0), reused 0 (delta 0), pack-reused 226
Receiving objects: 100% (226/226), 144.02 MiB | 37.09 MiB/s, done.
Resolving deltas: 100% (29/29), done.


In [10]:
### Download Fasttext pre-computed embbedings
##### ATENTION, LANGS ARE HARD CODED HERE, FOR NEW LANGUAGES: CHANGE #####
hardCodedLangs = ['ru', 'ja', 'fr', 'es', 'en', 'ar']
for l in hardCodedLangs:
    url= 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.%s.vec' % l
    !wget $url

--2018-08-23 22:40:23--  https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.ru.vec
Resolving webproxy.eqiad.wmnet (webproxy.eqiad.wmnet)... 2620:0:861:1:208:80:154:22, 208.80.154.22
Connecting to webproxy.eqiad.wmnet (webproxy.eqiad.wmnet)|2620:0:861:1:208:80:154:22|:8080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 4954492872 (4.6G) [binary/octet-stream]
Saving to: ‘wiki.ru.vec.1’

wiki.ru.vec.1       100%[===================>]   4.61G  34.2MB/s    in 7m 53s  

2018-08-23 22:48:17 (9.98 MB/s) - ‘wiki.ru.vec.1’ saved [4954492872/4954492872]

--2018-08-23 22:48:17--  https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.ja.vec
Resolving webproxy.eqiad.wmnet (webproxy.eqiad.wmnet)... 2620:0:861:1:208:80:154:22, 208.80.154.22
Connecting to webproxy.eqiad.wmnet (webproxy.eqiad.wmnet)|2620:0:861:1:208:80:154:22|:8080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 1374192861 (1.3G) [binary/octet-stream]
Saving to: ‘wiki.ja.vec’

wiki.j

## Compute distances 

In [1]:
def fasttextDistance(sec1,sec2,vectorsLang1,vectorsLang2):
    '''
    Take two sections, create a vector for each of them summing all the words
    return cosine similarity
    '''
    if (sec1!='') & (sec2!=''): #to avoid '' empty sections
        sec1 = sec1.lower().split()
        sec2 = sec2.lower().split()
        sec1Vector  = sum([vectorsLang1[word] for word in sec1 if word in vectorsLang1])/len(sec1)
        sec2Vector  = sum([vectorsLang2[word] for word in sec2 if word in vectorsLang2])/len(sec2)
        distance  = vectorsLang2.cosine_similarity(sec1Vector,sec2Vector)
        if not isinstance(distance,float): #when at least one of the sections is not the vectorial space, the result is 'nan'
            return 0
        else:
            return distance
    else:
        return 0

In [2]:
import glob,os
import os
from itertools import product
from fastText_multilingual.fasttext import FastVector
import pandas as pd

output = []

vectors = sorted(glob.glob('SectionAligmentData/wiki.*.vec'), key=os.path.getsize) #sorted by size to load the largest files just once
lang2 = ''
while vectors:
    lang1 = vectors.pop()
    if lang1 == lang2:
        lang1_dictionary = lang2_dictionary
    else:
        %time lang1_dictionary = FastVector(vector_file=lang1)
    lang1_code = lang1.split('.')[1]
    lang1_dictionary.apply_transform('fastText_multilingual/alignment_matrices/%s.txt' % lang1_code)
    #secsLang1 = pd.read_csv('SectionAligmentData/%s.tsv' % lang1_code,'\t')['source_title'].unique()
    for lang2 in vectors:
        lang2_dictionary = FastVector(vector_file=lang2)
        lang2_code = lang2.split('.')[1]
        lang2_dictionary.apply_transform('fastText_multilingual/alignment_matrices/%s.txt' % lang2_code)
        secsLang2 = pd.read_csv('SectionAligmentData/%s.tsv' % lang2_code,'\t')['source_title'].unique()
        try:
            secPairs = pd.read_pickle('resultsMapping-CoOcurrenceCountPandas/mapping_%s_%s.p' % (lang1_code,lang2_code))
        except:
            secPairs = pd.read_pickle('resultsMapping-CoOcurrenceCountPandas/mapping_%s_%s.p' % (lang2_code,lang1_code))
        del(secPairs['count']) # we dont use the co ocurrence count.
        secPairs['Distance'] = secPairs.apply(lambda row: fasttextDistance(row[lang1_code],row[lang2_code],lang1_dictionary,lang2_dictionary),axis=1)
        secPairs.to_pickle('FastTextDistances/distances_%s_%s.p' %  (lang1_code,lang2_code))
        


reading word vectors from SectionAligmentData/wiki.en.vec
CPU times: user 2min 58s, sys: 6.32 s, total: 3min 5s
Wall time: 3min 4s
reading word vectors from SectionAligmentData/wiki.ja.vec


/srv/home/dsaez/fastText_multilingual/fasttext.py:144: RuntimeWarning: invalid value encountered in double_scalars
  (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))
/srv/home/dsaez/fastText_multilingual/fasttext.py:144: RuntimeWarning: invalid value encountered in true_divide
  (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))


reading word vectors from SectionAligmentData/wiki.ar.vec


/srv/home/dsaez/fastText_multilingual/fasttext.py:144: RuntimeWarning: invalid value encountered in true_divide
  (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))
/srv/home/dsaez/fastText_multilingual/fasttext.py:144: RuntimeWarning: invalid value encountered in double_scalars
  (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))


reading word vectors from SectionAligmentData/wiki.es.vec


...

/srv/home/dsaez/fastText_multilingual/fasttext.py:144: RuntimeWarning: invalid value encountered in true_divide
  (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))
/srv/home/dsaez/fastText_multilingual/fasttext.py:144: RuntimeWarning: invalid value encountered in double_scalars
  (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))


reading word vectors from SectionAligmentData/wiki.ar.vec


/srv/home/dsaez/fastText_multilingual/fasttext.py:144: RuntimeWarning: invalid value encountered in true_divide
  (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))
/srv/home/dsaez/fastText_multilingual/fasttext.py:144: RuntimeWarning: invalid value encountered in double_scalars
  (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))


reading word vectors from SectionAligmentData/wiki.ja.vec


/srv/home/dsaez/fastText_multilingual/fasttext.py:144: RuntimeWarning: invalid value encountered in true_divide
  (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))
/srv/home/dsaez/fastText_multilingual/fasttext.py:144: RuntimeWarning: invalid value encountered in double_scalars
  (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))


In [3]:
x = lang1_dictionary

In [7]:
lang1_dictionary = '121'

In [8]:
x

In [27]:
        secPairs['Distance'] = secPairs.apply(lambda row: fasttextDistance(row[lang1_code],row[lang2_code],lang1_dictionary,lang2_dictionary),axis=1)


/srv/home/dsaez/fastText_multilingual/fasttext.py:144: RuntimeWarning: invalid value encountered in double_scalars
  (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))
/srv/home/dsaez/fastText_multilingual/fasttext.py:144: RuntimeWarning: invalid value encountered in true_divide
  (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))


In [25]:
secPairs.iloc[0] 


ja                               歴史
en    Noted people from Saiwai Ward
Name: 0, dtype: object

In [5]:
import pandas as pd
secPairs = pd.read_pickle('FastTextDistances/distances_%s_%s.p' % ('ru','fr'))

In [7]:
secPairs.sort_values('Distance',ascending=False)

,ru,fr,Distance
82456,Награды,[[Image:Judo pictogram.svg|30px]] Judo,0.258869
1208370,Награды,Judo,0.258869
1605044,История,Écluse et ascenseur à poissons,0.258089
1952552,Текст,Paroles et musique,0.255266
965743,История,Liste des comtes de Salona,0.250032
814764,Текст,Musique,0.249535
1590760,Олимпийские виды спорта,Bilan,0.249433
59172,Комментарии,Massacre,0.247148
1840108,Комментарии,Nationalisme,0.246348
565521,Критика,Église catholique romaine,0.244014
